In [1]:

import numpy as np
import pandas as pd
import torch
import os
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
from tqdm import tqdm, trange

import gc

gc.collect()

torch.cuda.empty_cache()

In [2]:
class MedTextDataset(Dataset):
    
    def __init__(self, path_file, control_code, truncate=False, gpt2_type="gpt2", max_length=768):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.comments = []
        
        with open(path_file) as file:
            comments = file.readlines()
            
            self.comments = [self.comment_to_tensor(c, control_code, max_length) for c in comments]
            
                
        if truncate:
            self.comments = self.comments[:20000]
        self.comments_count = len(self.comments)
        
        
    def comment_to_tensor(self, comment, control_code, max_length):
        return torch.tensor(self.tokenizer.encode(f"<|{control_code}|>{comment[:max_length]}<|EOS|>"))
        
    def __len__(self):
        return self.comments_count

    def __getitem__(self, item):
        return self.comments[item]

In [3]:
DATA_TRAIN_PRUEBA = '../tagged_files/train_tagged_prueba.txt'
train_p_dataset = MedTextDataset(DATA_TRAIN_PRUEBA, 'BOS')

In [4]:
len(train_p_dataset)

9

In [5]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [6]:
def train(
    dataset,
    model,
    batch_size=16,
    epochs=4,
    lr=2e-5,
    max_seq_len=400,
    warmup_steps=5000,
    gpt2_type="gpt2",
    device="cuda",
    output_dir=".",
    output_prefix="medtex",
    test_mode=False,
    save_model_on_epoch=False,
):

    acc_steps = 100

    torch.cuda.empty_cache()

    model = model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [7]:
gpt2_type = "gpt2"

In [8]:

model = train(
    train_p_dataset,
    GPT2LMHeadModel.from_pretrained(gpt2_type),
    batch_size=64,
    epochs=3,
    lr=2e-5,
    max_seq_len=140,
    warmup_steps=200,
    gpt2_type=gpt2_type,
    device="cuda",
    output_dir="trained_models",
    output_prefix="medtext",
    save_model_on_epoch=True
)

4it [00:00, 103.48it/s]Training epoch 0



RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 3.93 GiB total capacity; 1.95 GiB already allocated; 6.19 MiB free; 2.04 GiB reserved in total by PyTorch)